# Tagged Hashes
* Each hash is different so that hashes cannot feasibly collide
* There are 10 different contexts, each creating its own set of hashes
* The hash is SHA256, but with 64 bytes before the actual bytes being hashed
* The 64 bytes are another SHA256 of the tag (e.g. "BIP340/aux") repeated twice
* H_aux(x) = SHA256(SHA256("BIP340/aux") + SHA256("BIP340/aux") + x)


In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import hash
import op
import taproot

In [ ]:
# Example Tagged Hashes
from hash import sha256
challenge_tag = b"BIP0340/challenge"
msg = b"some message"
challenge_hash = sha256(challenge_tag)
hash_challenge = sha256(challenge_hash + challenge_hash + msg)
print(hash_challenge.hex())

### Exercise 1

What is the tagged hash "BIP0340/aux" of "hello world"?



In [ ]:
# Exercise 1

from hash import sha256
# define the challenge tag and the message
challenge_tag = b"BIP0340/aux"
msg = b"hello world"
# calculate the challenge tag hash using sha256
challenge_tag_hash = sha256(challenge_tag)
# calculate the hash of the challenge
hash_challenge = sha256(challenge_tag_hash + challenge_tag_hash + msg)
print(hash_challenge.hex())

### Exercise 2




#### Make [this test](/edit/session/hash.py) pass: `hash.py:HashTest:test_tagged_hash`

In [ ]:
# Exercise 2

reload(hash)
run(hash.HashTest('test_tagged_hash'))

# $x$-only keys
* Assume $y$ is even
* Serialized as 32-bytes
* The private key $e$ is flipped to $N-e$ if $y$ is odd
* $eG=P=(x,y)$ means $(N-e)G=0-eG=-P=(x,-y)$
* Lots of flipping!


In [ ]:
# Example X-only pubkey
from ecc import PrivateKey, S256Point
from helper import int_to_big_endian
pubkey = PrivateKey(12345).point
xonly = int_to_big_endian(pubkey.x.num, 32)
print(xonly.hex())
pubkey2 = S256Point.parse(xonly)
print(pubkey.xonly() == pubkey2.xonly())

### Exercise 3
Find the $x$-only pubkey format for the private key with the secret 21,000,000



In [ ]:
# Exercise 3

from ecc import PrivateKey
secret = 21000000
# create a private key with the secret
priv = PrivateKey(secret)
# get the public point for the private key
point = priv.point
# convert the x coordinate to a big-endian integer 32 bytes
xonly = int_to_big_endian(point.x.num, 32)
print(xonly.hex())

### Exercise 4




#### Make [this test](/edit/session/ecc.py) pass: `ecc.py:XOnlyTest:test_xonly`

In [ ]:
# Exercise 4

reload(ecc)
run(ecc.XOnlyTest('test_xonly'))

# Schnorr Verification
* $eG=P$, $m$ message, $kG=R$, $H$ is a hash function
* Signature is $(R,s)$ where $s=k + e H(R||P||m)$
* $$-H(R||P||m)P+sG \\ =-H(R||P||m)P+(k+e H(R||P||m))G \\ =-H(R||P||m)P+kG+H(R||P||m)(eG) \\ =R+H(R||P||m)P-H(R||P||m)P=R$$


In [ ]:
from ecc import S256Point, SchnorrSignature, G, N
from helper import sha256, big_endian_to_int
from hash import hash_challenge
# the message we're signing
msg = sha256(b"I attest to understanding Schnorr Signatures")
# the signature we're using
sig_raw = bytes.fromhex("f3626c99fe36167e5fef6b95e5ed6e5687caa4dc828986a7de8f9423c0f77f9bc73091ed86085ce43de0e255b3d0afafc7eee41ddc9970c3dc8472acfcdfd39a")
sig = SchnorrSignature.parse(sig_raw)
# the pubkey we are using
xonly = bytes.fromhex("f01d6b9018ab421dd410404cb869072065522bf85734008f105cf385a023a80f")
point = S256Point.parse(xonly)
# calculate the commitment which is R || P || msg
commitment = sig.r.xonly() + point.xonly() + msg
# hash_challenge the commitment, interpret as big endian and mod by N
challenge = big_endian_to_int(hash_challenge(commitment)) % N
# the target is the -challenge * point + s * G
target = -challenge * point + sig.s * G
print(target == sig.r)

### Exercise 5

Verify this Schnorr Signature

Pubkey = cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91
Signature = 2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4
Message = 1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612



In [ ]:
# Exercise 5

from ecc import SchnorrSignature, S256Point, N, G
from hash import hash_challenge
from helper import big_endian_to_int
p_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
pubkey = S256Point.parse(p_raw)
sig_raw = bytes.fromhex("2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4")
sig = SchnorrSignature.parse(sig_raw)
msg = bytes.fromhex("1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612")
# create the commitment: R || P || m (points should be xonly)
commitment = sig.r.xonly() + pubkey.xonly() + msg
# hash the commitment with hash_challenge and then big_endian_to_int it
h = big_endian_to_int(hash_challenge(commitment))
# check that -hP+sG=R
print(-h*pubkey + sig.s*G  == sig.r)

### Exercise 6




#### Make [this test](/edit/session/ecc.py) pass: `ecc.py:SchnorrTest:test_verify`

In [ ]:
# Exercise 6

reload(ecc)
run(ecc.SchnorrTest('test_verify'))

# Schnorr Signing
* $eG=P$, $m$ message, $k$ random
* $kG=R$, $H$ is a hash function
* $s=k+e H(R||P||m)$ where $R$ and $P$ are $x$-only
* Signature is $(R,s)$


In [ ]:
# Example Signing
from ecc import PrivateKey, N, G
from helper import sha256, big_endian_to_int
priv = PrivateKey(12345)
if priv.point.y.num % 2 == 1:
    d = N - priv.secret
else:
    d = priv.secret
msg = sha256(b"I attest to understanding Schnorr Signatures")
k = 21016020145315867006318399104346325815084469783631925097217883979013588851039
r = k * G
if r.y.num % 2 == 1:
    k = N - k
    r = k * G
commitment = r.xonly() + priv.point.xonly() + msg
e = big_endian_to_int(hash_challenge(commitment)) % N
s = (k + e * d) % N
sig = SchnorrSignature(r, s)
if not priv.point.verify_schnorr(msg, sig):
    raise RuntimeError("Bad Signature")
print(sig.serialize().hex())

### Exercise 7

Sign the message b"I'm learning Taproot!" with the private key 21,000,000



In [ ]:
# Exercise 7

from ecc import PrivateKey, N, G
from helper import sha256
# create the private key
priv = PrivateKey(21000000)
# calculate d (working secret) based on whether the y is even or odd
if priv.point.y.num % 2 == 1:
    d = N - priv.secret
else:
    d = priv.secret
# create the message
msg = sha256(b"I'm learning Taproot!")
# We'll learn more about k later, for now use 987654321
k = 987654321
# get the resulting R=kG point
r = k * G
# if R's y coordinate is odd, flip the k
if r.y.num % 2 == 1:
    # set k to N - k
    k = N - k
    # recalculate R
    r = k * G
# calculate the commitment which is: R || P || msg
commitment = r.xonly() + priv.point.xonly() + msg
# hash_challenge the result and interpret as a big endian integer mod the result by N and this is your e
e = big_endian_to_int(hash_challenge(commitment)) % N
# calculate s which is (k+ed) mod N
s = (k + e * d) % N
# create a SchnorrSignature object using the R and s
sig = SchnorrSignature(r, s)
# check that this schnorr signature verifies
if not priv.point.verify_schnorr(msg, sig):
    raise RuntimeError("Bad Signature")
# print the serialized hex of the signature
print(sig.serialize().hex())

### Exercise 8




#### Make [this test](/edit/session/ecc.py) pass: `ecc.py:SchnorrTest:test_sign`

In [ ]:
# Exercise 8

reload(ecc)
run(ecc.SchnorrTest('test_sign'))

# Batch Verification
* $e_iG=P_i$, $m_i$ message, $H$
* Signature is $(R_i,s_i)$, $h_i=H(R_i||P_i||m_i)$
* $-h_i P_1+s_1G=R_1$
* $-h_i P_2+s_2G=R_2$
* $-h_1 P_1-h_2 P_1+(s_1+s_2)G=R_1+R_2$
* $(s_1+s_2)G=R_1+R_2+h_1 P_1+h_2 P_2$


### Exercise 9

Batch Verify these two Schnorr Signatures

Pubkey 1 = cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91
Pubkey 2 = e79c4eb45764bd015542f6779cc70fef44b7a2432f839264768288efab886291

Signature 1 = 2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4
Signature 2 = b6e52f38bc24f1420c4fdae8fa0f04b9b0374a12f18fd4699b06df53eb1386bfa88c1835cd19470cf8c76550eb549c988f9c8fac00cc56fadd4fcc3bf9d8800e

Message 1 = 1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612
Message 2 = af1c325abcb0cced3a4166ce67be1db659ae1dd574fe49b0f2941d8d4882d62c



In [ ]:
# Exercise 9

from ecc import SchnorrSignature, S256Point, N, G
from hash import hash_challenge
from helper import big_endian_to_int
p1_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
p2_raw = bytes.fromhex("e79c4eb45764bd015542f6779cc70fef44b7a2432f839264768288efab886291")
p1 = S256Point.parse(p1_raw)
p2 = S256Point.parse(p2_raw)
sig1_raw = bytes.fromhex("2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4")
sig2_raw = bytes.fromhex("b6e52f38bc24f1420c4fdae8fa0f04b9b0374a12f18fd4699b06df53eb1386bfa88c1835cd19470cf8c76550eb549c988f9c8fac00cc56fadd4fcc3bf9d8800e")
sig1 = SchnorrSignature.parse(sig1_raw)
sig2 = SchnorrSignature.parse(sig2_raw)
msg1 = bytes.fromhex("1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612")
msg2 = bytes.fromhex("af1c325abcb0cced3a4166ce67be1db659ae1dd574fe49b0f2941d8d4882d62c")
# define s as the s_i sum (make sure to mod by N)
s = (sig1.s + sig2.s) % N
# define r as the signatures' r sum
r = sig1.r + sig2.r
# create the commitments: R_i||P_i||m_i
commitment_1 = sig1.r.xonly() + p1.xonly() + msg1
commitment_2 = sig2.r.xonly() + p2.xonly() + msg2
# define the h's as the hash_challenge of the commitment as a big endian integer
h1 = big_endian_to_int(hash_challenge(commitment_1))
h2 = big_endian_to_int(hash_challenge(commitment_2))
# compute the sum of the h_i P_i's
h = h1*p1 + h2*p2
# check that sG=R+h
print(s*G == r+h)

# How to spend from the KeyPath
* You have to know the Merkle Root of the ScriptPath
* The internal public key is hashed together with the Merkle Root to generate the tweak $t$
* The formula is $t=H(P||t)$ where H is a Tagged Hash (TapTweak)
* $Q=P+tG$, and $eG=P$ which means $Q=eG+tG$ and $Q=(e+t)G$
* $e+t$ is your private key, which can sign for the public key Q
* Witness only needs the Schnorr Signature
* If you don't want a script path, $t$ is just empty


In [ ]:
# Example Q calculation for a single-key
from ecc import S256Point, G
from hash import hash_taptweak
from helper import big_endian_to_int
from script import P2TRScriptPubKey
internal_pubkey_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
internal_pubkey = S256Point.parse(internal_pubkey_raw)
tweak = big_endian_to_int(hash_taptweak(internal_pubkey_raw))
external_pubkey = internal_pubkey + tweak * G
script_pubkey = P2TRScriptPubKey(external_pubkey)
print(script_pubkey)

### Exercise 10

Make a P2TR ScriptPubKey using the private key 9284736473



In [ ]:
# Exercise 10

from ecc import PrivateKey, G
from hash import hash_taptweak
from helper import big_endian_to_int
from script import P2TRScriptPubKey
priv = PrivateKey(9284736473)
# get the internal pubkey
internal_pubkey = priv.point
# calculate the tweak
tweak = big_endian_to_int(hash_taptweak(internal_pubkey.xonly()))
# Q = P + tG
external_pubkey = internal_pubkey + tweak * G
# use P2TRScriptPubKey to create the ScriptPubKey
script_pubkey = P2TRScriptPubKey(external_pubkey)
# print the ScriptPubKey
print(script_pubkey)

### Exercise 11




#### Make [this test](/edit/session/ecc.py) pass: `ecc.py:TapRootTest:test_default_tweak`

In [ ]:
# Exercise 11

reload(ecc)
run(ecc.TapRootTest('test_default_tweak'))

### Exercise 12




#### Make [this test](/edit/session/ecc.py) pass: `ecc.py:TapRootTest:test_tweaked_key`

In [ ]:
# Exercise 12

reload(ecc)
run(ecc.TapRootTest('test_tweaked_key'))

### Exercise 13




#### Make [this test](/edit/session/ecc.py) pass: `ecc.py:TapRootTest:test_p2tr_script`

In [ ]:
# Exercise 13

reload(ecc)
run(ecc.TapRootTest('test_p2tr_script'))

# P2TR Addresses
* Uses Bech32m, which is different than Bech32 (BIP350)
* Segwit v0 uses Bech32
* Taproot (Segwit v1) uses Bech32m
* Has error correcting capability and uses 32 letters/numbers
* Segwit v0 addresses start with bc1q and p2wpkh is shorter than p2wsh
* Segwit v1 addresses start with bc1p and they're all one length


In [ ]:
# Example of getting a p2tr address
from ecc import S256Point
internal_pubkey_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
internal_pubkey = S256Point.parse(internal_pubkey_raw)
print(internal_pubkey.p2tr_address())
print(internal_pubkey.p2tr_address(network="signet"))

### Exercise 14

Make your own Signet P2TR Address

Write down your address at [this link](https://docs.google.com/spreadsheets/d/1wUNeR-g5qY_2lh18gxg5JIQr352fOW2wZia_QErj4V0/edit?usp=sharing) under "keypath address"



In [ ]:
# Exercise 14

from ecc import PrivateKey
from helper import sha256
my_email = b"jimmy@programmingblockchain.com"
my_secret = big_endian_to_int(sha256(my_email))
# create the private key object
priv = PrivateKey(my_secret)
# get the public point
point = priv.point
# print the p2tr_address with network set to "signet"
print(point.p2tr_address(network="signet"))

# Spending plan
* We have 20,000 sats in this output: 871864d7631024465fc210e553fa9f50e7f0f2359288ad121aa733d65e366995:0
* We want to spend all of it to tb1ptaqplrhnyh3kq85n7dtm5vcpgstt0ev80f4wd8ngeppch4fzu8mquchufq
* 1 input/1 output transaction


In [ ]:
# Spending from a p2tr
from ecc import PrivateKey, N
from helper import sha256, big_endian_to_int
from script import address_to_script_pubkey
from tx import Tx, TxIn, TxOut
my_email = b"jimmy@programmingblockchain.com"
my_secret = big_endian_to_int(sha256(my_email))
priv = PrivateKey(my_secret)
prev_tx = bytes.fromhex("871864d7631024465fc210e553fa9f50e7f0f2359288ad121aa733d65e366995")
prev_index = 0
target_address = "tb1ptaqplrhnyh3kq85n7dtm5vcpgstt0ev80f4wd8ngeppch4fzu8mquchufq"
fee = 500
tx_in = TxIn(prev_tx, prev_index)
target_script_pubkey = address_to_script_pubkey(target_address)
target_amount = tx_in.value(network="signet") - fee
tx_out = TxOut(target_amount, target_script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], network="signet", segwit=True)
tweaked_secret = (priv.secret + big_endian_to_int(priv.point.tweak())) % N
tweaked_key = PrivateKey(tweaked_secret)
tx_obj.sign_p2tr_keypath(0, tweaked_key)
print(tx_obj.serialize().hex())

### Exercise 15




#### Make [this test](/edit/session/ecc.py) pass: `ecc.py:PrivateKeyTest:test_tweaked_key`

In [ ]:
# Exercise 15

reload(ecc)
run(ecc.PrivateKeyTest('test_tweaked_key'))

### Exercise 16

## Checkpoint Exercise

You have been sent 100,000 sats to your address on Signet. Send 40,000 sats back to <code>tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg</code>, the rest to yourself.

Use <a href="https://mempool.space/signet/tx/push to broadcast your transaction" target="_mempool">Mempool Signet</a> to broadcast your transaction



In [ ]:
# Exercise 16

from ecc import PrivateKey
from helper import sha256
from script import address_to_script_pubkey
from tx import Tx, TxIn, TxOut
my_email = b"jimmy@programmingblockchain.com"
my_secret = big_endian_to_int(sha256(my_email))
priv = PrivateKey(my_secret)
prev_tx = bytes.fromhex("25096348891ff6b120b88c944501791f8809698474569cc994d63dc5bcfe6a37")
target_address = "tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg"
target_amount = 40000
fee = 500
# fill this in from the block explorer
prev_index = 0
# create the one input
tx_in = TxIn(prev_tx, prev_index)
# use the address_to_script_pubkey to get the ScriptPubKey
target_script_pubkey = address_to_script_pubkey(target_address)
# create the target output
tx_out_1 = TxOut(target_amount, target_script_pubkey)
# calculate the change amount
change_amount = 100000 - target_amount - fee
# use the private key's point's p2tr_script method to get the change ScriptPubkey
change_script_pubkey = priv.point.p2tr_script()
# create the change output
tx_out_2 = TxOut(change_amount, change_script_pubkey)
# create the transaction
tx_obj = Tx(1, [tx_in], [tx_out_1, tx_out_2], network="signet", segwit=True)
# sign the transaction using the tweaked key and the sign_p2tr_keypath method
tx_obj.sign_p2tr_keypath(0, priv.tweaked_key())
# print the serialized hex
print(tx_obj.serialize().hex())

# OP_CHECKSIGADD
* Consumes the top three elements: a pubkey, a number, and a signature.
* Valid sig, returns the number+1 to the stack
* Invalid sig, returns the number back to the stack


### Exercise 17




#### Make [this test](/edit/session/op.py) pass: `op.py:TapScriptTest:test_opchecksigadd`

In [ ]:
# Exercise 17

reload(op)
run(op.TapScriptTest('test_opchecksigadd'))

# Example TapScripts
* 1-of-1 (pay-to-pubkey) [pubkey, OP_CHECKSIG]
* 2-of-2 [pubkey A, OP_CHECKSIGVERIFY, pubkey B, OP_CHECKSIG]
* 2-of-3 [pubkey A, OP_CHECKSIG, pubkey B, OP_CHECKSIGADD, pubkey C, OP_CHECKSIGADD, OP_2, OP_EQUAL]
* halvening timelock 1-of-1 [840000, OP_CHECKLOCKTIMEVERIFY, OP_DROP, pubkey, OP_CHECKSIG]


In [ ]:
# Example TapScripts
from ecc import PrivateKey
from op import encode_minimal_num
from taproot import TapScript
pubkey_a = PrivateKey(11111111).point.xonly()
pubkey_b = PrivateKey(22222222).point.xonly()
pubkey_c = PrivateKey(33333333).point.xonly()
# 1-of-1 (0xAC is OP_CHECKSIG)
tap_script = TapScript([pubkey_a, 0xAC])
print(tap_script)
# 2-of-2 (0xAD is OP_CHECKSIGVERIFY)
tap_script = TapScript([pubkey_a, 0xAD, pubkey_b, 0xAC])
print(tap_script)
# 2-of-3 (0xBA is OP_CHECKSIGADD, 0x52 is OP_2, 0x87 is OP_EQUAL)
tap_script = TapScript([pubkey_a, 0xAD, pubkey_b, 0xBA, pubkey_c, 0xBA, 0x52, 0x87])
print(tap_script)
# halvening timelock 1-of-1 (0xB1 is OP_CLTV, 0x75 is OP_DROP)
tap_script = TapScript([encode_minimal_num(840000), 0xB1, 0x75, pubkey_a, 0xAC])
print(tap_script)

### Exercise 18

Make a TapScript for 4-of-4 using pubkeys from private keys which correspond to 10101, 20202, 30303, 40404



In [ ]:
# Exercise 18

from ecc import PrivateKey
from taproot import TapScript
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create a 4-of-4 tapscript that uses OP_CHECKSIGVERIFY (0xad) and OP_CHECKSIG (0xac)
tap_script = TapScript([pubkey_1, 0xAD, pubkey_2, 0xAD, pubkey_3, 0xAD, pubkey_4, 0xAC])
# print the TapScript
print(tap_script)

# TapLeaf
* These are the leaves of the Merkle Tree
* Has a TapLeaf Version (<code>0xc0</code>) and TapScript
* Any Leaf can be executed to satisfy the Taproot Script Path
* Hash of a TapLeaf is a Tagged Hash (TapLeaf) of the version + TapScript


In [ ]:
# Example of making a TapLeaf and calculating the hash
from ecc import PrivateKey
from hash import hash_tapleaf
from helper import int_to_byte
from taproot import TapScript, TapLeaf
pubkey_a = PrivateKey(11111111).point.xonly()
pubkey_b = PrivateKey(22222222).point.xonly()
tap_script = TapScript([pubkey_a, 0xAD, pubkey_b, 0xAC])
tap_leaf = TapLeaf(tap_script)
h = hash_tapleaf(int_to_byte(tap_leaf.tapleaf_version) + tap_leaf.tap_script.serialize())
print(h.hex())

### Exercise 19

Calculate the TapLeaf hash whose TapScript is a 2-of-4 using pubkeys from private keys which correspond to 10101, 20202, 30303, 40404



In [ ]:
# Exercise 19

from ecc import PrivateKey
from hash import hash_tapleaf
from helper import int_to_byte
from taproot import TapScript, TapLeaf
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create a 2-of-4 TapScript that uses OP_CHECKSIG (0xac), OP_CHECKSIGADD (0xba), OP_2 (0x52) and OP_EQUAL (0x87)
tap_script = TapScript([pubkey_1, 0xAC, pubkey_2, 0xBA, pubkey_3, 0xBA, pubkey_4, 0xBA, 0x52, 0x87])
# create the TapLeaf with the TapScript
tap_leaf = TapLeaf(tap_script)
# calculate the hash
h = hash_tapleaf(int_to_byte(tap_leaf.tapleaf_version) + tap_leaf.tap_script.serialize())
# print the hash hex
print(h.hex())

### Exercise 20




#### Make [this test](/edit/session/taproot.py) pass: `taproot.py:TapRootTest:test_tapleaf_hash`

In [ ]:
# Exercise 20

reload(taproot)
run(taproot.TapRootTest('test_tapleaf_hash'))

# TapBranch
* These are the branches of the Merkle Tree
* These connect a left child and a right child.
* Each child is a TapLeaf or TapBranch
* Hash of a TapBranch is a Tagged Hash (TapBranch) of the left hash and right hash, sorted


In [ ]:
from ecc import PrivateKey
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
pubkey_1 = PrivateKey(11111111).point.xonly()
pubkey_2 = PrivateKey(22222222).point.xonly()
tap_script_1 = TapScript([pubkey_1, 0xAC])
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_leaf_2 = TapLeaf(tap_script_2)
tap_branch = TapBranch(tap_leaf_1, tap_leaf_2)
left_hash = tap_branch.left.hash()
right_hash = tap_branch.right.hash()
if left_hash > right_hash:
    h = hash_tapbranch(left_hash + right_hash)
else:
    h = hash_tapbranch(right_hash + left_hash)
print(h.hex())

### Exercise 21

TabBranch Calculation

Calculate the TabBranch hash whose left and right nodes are TapLeafs whose TapScripts are for a 1-of-2 using pubkeys from private keys which correspond to (10101, 20202) for the left, (30303, 40404) for the right



In [ ]:
# Exercise 21

from ecc import PrivateKey
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create two 1-of-2 TapScripts
tap_script_1 = TapScript([pubkey_1, 0xAC, pubkey_2, 0xBA, 0x51, 0x87])
tap_script_2 = TapScript([pubkey_3, 0xAC, pubkey_4, 0xBA, 0x51, 0x87])
# create two TapLeafs with the TapScripts
tap_leaf_1 = TapLeaf(tap_script_1)
tap_leaf_2 = TapLeaf(tap_script_2)
# create the branch
tap_branch = TapBranch(tap_leaf_1, tap_leaf_2)
# get the left and right hashes
left_hash = tap_branch.left.hash()
right_hash = tap_branch.right.hash()
# calculate the hash using the sorted order with hash_tapbranch
if left_hash > right_hash:
    h = hash_tapbranch(left_hash + right_hash)
else:
    h = hash_tapbranch(right_hash + left_hash)
# print the hex of the hash
print(h.hex())

### Exercise 22




#### Make [this test](/edit/session/taproot.py) pass: `taproot.py:TapRootTest:test_tapbranch_hash`

In [ ]:
# Exercise 22

reload(taproot)
run(taproot.TapRootTest('test_tapbranch_hash'))

# Computing the Merkle Root
* The Merkle Root is the hash of the root element of the Merkle Tree
* For TapLeaf: Tagged hash (TapLeaf) of TapLeaf Version followed by the TapScript
* For TapBranch: Tagged hash (TapBranch) of the sorted children (left and right)
* It doesn't have to be the hash of anything, just has to be 32 bytes


In [ ]:
# Example of Comupting the Merkle Root
from ecc import PrivateKey
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
pubkey_1 = PrivateKey(11111111).point.xonly()
pubkey_2 = PrivateKey(22222222).point.xonly()
pubkey_3 = PrivateKey(33333333).point.xonly()
tap_script_1 = TapScript([pubkey_1, 0xAC])
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_leaf_2 = TapLeaf(tap_script_2)
tap_leaf_3 = TapLeaf(tap_script_3)
tap_branch_1 = TapBranch(tap_leaf_1, tap_leaf_2)
tap_root = TapBranch(tap_branch_1, tap_leaf_3)
merkle_root = tap_root.hash()
print(merkle_root.hex())

### Exercise 23

Calculate the External PubKey for a Taproot output whose internal pubkey is 90909 and whose Merkle Root is from two TapBranches, each of which is a single signature TapLeaf. The private keys corresponding to the left TapBranch's TapLeafs are 10101 and 20202. The private keys corresponding to the right TapBranch's TapLeafs are 30303 and 40404.



In [ ]:
# Exercise 23

from ecc import PrivateKey
from helper import big_endian_to_int
from taproot import TapScript, TapLeaf, TapBranch
internal_pubkey = PrivateKey(90909).point
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create four tap scripts one for each pubkey
tap_script_1 = TapScript([pubkey_1, 0xAC])
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_script_4 = TapScript([pubkey_4, 0xAC])
# create four TapLeafs with the TapScripts
tap_leaf_1 = TapLeaf(tap_script_1)
tap_leaf_2 = TapLeaf(tap_script_2)
tap_leaf_3 = TapLeaf(tap_script_3)
tap_leaf_4 = TapLeaf(tap_script_4)
# create two TapBranches that have these TapLeafs
tap_branch_1 = TapBranch(tap_leaf_1, tap_leaf_2)
tap_branch_2 = TapBranch(tap_leaf_3, tap_leaf_4)
# create another TapBranch that corresponds to the merkle root and get its hash
merkle_root = TapBranch(tap_branch_1, tap_branch_2).hash()
# the external public key is the internal public key tweaked with the Merkle Root
external_pubkey = internal_pubkey.tweaked_key(merkle_root)
# print the hex of the xonly of the external pubkey
print(external_pubkey.xonly().hex())

# Control Block
* Required for spending a TapScript, last element of Witness
* TapScript Version (<code>0xc0</code> or <code>0xc1</code>)
* The last bit expresses the parity of the external pubkey, which is necessary for batch verification
* Internal PubKey $P$
* Merkle Proof (list of hashes to combine to get to the Merkle Root)


# Merkle Proof
* List of hashes
* Combine each with the hash of the TapScript, sorting them each time
* The result is the Merkle Root, which can be combined with the Internal PubKey $P$ to get the tweak $t$
* If the result of $P+tG=Q$ where $Q$ is the External PubKey from the UTXO, this is a valid TapScript


In [ ]:
# Example of Control Block Validation
from ecc import PrivateKey, S256Point
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
external_pubkey_xonly = bytes.fromhex("cbe433288ae1eede1f24818f08046d4e647fef808cfbbffc7d10f24a698eecfd")
pubkey_2 = bytes.fromhex("027aa71d9cdb31cd8fe037a6f441e624fe478a2deece7affa840312b14e971a4")
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_2 = TapLeaf(tap_script_2)
tap_leaf_1_hash = bytes.fromhex("76f5c1cdfc8b07dc8edca5bef2b4991201c5a0e18b1dbbcfe00ef2295b8f6dff")
tap_leaf_3_hash = bytes.fromhex("5dd270ec91aa5644d907059400edfd98e307a6f1c6fe3a2d1d4550674ff6bc6e")
internal_pubkey = S256Point.parse(bytes.fromhex("407910a4cfa5fe195ad4844b6069489fcb429f27dff811c65e99f7d776e943e5"))
current = tap_leaf_2.hash()
for h in (tap_leaf_1_hash, tap_leaf_3_hash):
    if h < current:
        current = hash_tapbranch(h + current)
    else:
        current = hash_tapbranch(current + h)
print(internal_pubkey.tweaked_key(current).xonly() == external_pubkey_xonly)
print(internal_pubkey.p2tr_address(current, network="signet"))

### Exercise 24

Validate the Control Block for the pubkey whose private key is 40404 for the previous external pubkey



In [ ]:
# Exercise 24

from ecc import PrivateKey, S256Point
from helper import big_endian_to_int
from taproot import TapScript, TapLeaf, TapBranch
external_pubkey_xonly = bytes.fromhex("8b9f09cd4a33e62b0c9d086056bbdeb7a218c1e4830291b9be56841b31d94ccb")
internal_pubkey = PrivateKey(90909).point
hash_1 = bytes.fromhex("22cac0b60bc7344152a8736425efd62532ee4d83e3de473ed82a64383b4e1208")
hash_2 = bytes.fromhex("a41d343d7419b99bfe8e66752fc3c45fd14aa2cc5ef5bf9073ed28dfc60e2e34")
pubkey_4 = bytes.fromhex("9e5f5a5c29d33c32185a3dc0a9ccb3e72743744dd869dd40b6265a23fd84a402")
# create the TapScript and TapLeaf for pubkey 4
tap_script_4 = TapScript([pubkey_4, 0xAC])
tap_leaf_4 = TapLeaf(tap_script_4)
# set the current hash to the TapLeaf's hash
current = tap_leaf_4.hash()
# loop through hash_1 and hash_2
for h in (hash_1, hash_2):
    # do a hash_tapbranch of h and current, sorted alphabetically
    if h < current:
        current = hash_tapbranch(h + current)
    else:
        current = hash_tapbranch(current + h)
# get the external pubkey using the current hash as the merkle root with the internal pubkey
external_pubkey = internal_pubkey.tweaked_key(current)
# check to see if the external pubkey's xonly is correct
print(external_pubkey.xonly() == external_pubkey_xonly)

### Exercise 25




#### Make [this test](/edit/session/taproot.py) pass: `taproot.py:TapRootTest:test_control_block`

In [ ]:
# Exercise 25

reload(taproot)
run(taproot.TapRootTest('test_control_block'))

### Exercise 26

Create a Signet P2TR address with these Script Spend conditions:

1. Internal Public Key is <code>cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e</code>
2. Leaf 1 and Leaf 2 make Branch 1, Branch 1 and Leaf 3 make Branch 2, which is the Merkle Root
3. All TapLeaf are single key locked TapScripts (pubkey, OP_CHECKSIG)
4. Leaf 1 uses your xonly pubkey
5. Leaf 2 uses this xonly pubkey: <code>331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec</code>
6. Leaf 3 uses this xonly pubkey: <code>158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f</code>



In [ ]:
# Exercise 26

from ecc import PrivateKey, S256Point
from helper import sha256, big_endian_to_int
from taproot import TapScript, TapLeaf, TapBranch
my_email = b"jimmy@programmingblockchain.com"
my_secret = big_endian_to_int(sha256(my_email))
internal_pubkey = S256Point.parse(bytes.fromhex("cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e"))
pubkey_2 = bytes.fromhex("331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec")
pubkey_3 = bytes.fromhex("158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f")
# get your xonly pubkey
my_xonly = PrivateKey(my_secret).point.xonly()
# make the first TapScript and TapLeaf using your xonly and OP_CHECKSIG (0xAC)
tap_script_1 = TapScript([my_xonly, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
# make the second and third TapLeaves using pubkey_2 and pubkey_3 respectively
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_2 = TapLeaf(tap_script_2)
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_leaf_3 = TapLeaf(tap_script_3)
# make a TapBranch with leaf 1 and 2
tap_branch_1 = TapBranch(tap_leaf_1, tap_leaf_2)
# make a TapBranch with branch 1 and leaf 3
tap_branch_2 = TapBranch(tap_branch_1, tap_leaf_3)
# get the hash of this branch, this is the Merkle Root
merkle_root = tap_branch_2.hash()
# print the address using the p2tr_address method of internal_pubkey and specify signet
print(internal_pubkey.p2tr_address(merkle_root, network="signet"))

### Exercise 27

Send yourself the rest of the coins from the output of the previous exercise to the address you just created

Use <a href="https://mempool.space/signet/tx/push to broadcast your transaction" target="_mempool">Mempool Signet</a> to broadcast your transaction



In [ ]:
# Exercise 27

from ecc import PrivateKey, S256Point
from helper import sha256, big_endian_to_int
from script import address_to_script_pubkey
from taproot import TapScript, TapLeaf, TapBranch
from tx import Tx, TxIn, TxOut
my_email = b"jimmy@programmingblockchain.com"
my_secret = big_endian_to_int(sha256(my_email))
my_private_key = PrivateKey(my_secret)
prev_tx = bytes.fromhex("69804495c439176266c473081e2f9a3cd298e60a17c8b035fd3070073b865a9c")
prev_index = 1
target_address = "tb1pxh7kypwsvxnat0z6588pufhx43r2fnqjyn846qj5kx8mgqcamvjsyn5cjg"
fee = 500
# create a transaction input
tx_in = TxIn(prev_tx, prev_index)
# calculate the target amount
target_amount = tx_in.value(network="signet") - fee
target_script = address_to_script_pubkey(target_address)
# create a transaction output
tx_out = TxOut(target_amount, target_script)
# create a transaction, segwit=True and network="signet"
tx_obj = Tx(1, [tx_in], [tx_out], network="signet", segwit=True)
# calculate the tweaked key from your private key
signing_key = my_private_key.tweaked_key()
# sign the transaction using sign_p2tr_keypath
tx_obj.sign_p2tr_keypath(0, signing_key)
# print the serialized hex
print(tx_obj.serialize().hex())

### Exercise 28

Now spend this output using the script path from the second TapLeaf send it all to <code>tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg</code>

Use <a href="https://mempool.space/signet/tx/push to broadcast your transaction" target="_mempool">Mempool Signet</a> to broadcast your transaction



In [ ]:
# Exercise 28

from ecc import PrivateKey, S256Point
from helper import sha256, big_endian_to_int
from script import address_to_script_pubkey
from taproot import TapScript, TapLeaf, TapBranch
from tx import Tx, TxIn, TxOut
from witness import Witness
my_email = b"jimmy@programmingblockchain.com"
my_secret = big_endian_to_int(sha256(my_email))
my_private_key = PrivateKey(my_secret)
internal_pubkey = S256Point.parse(bytes.fromhex("cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e"))
pubkey_2 = bytes.fromhex("331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec")
pubkey_3 = bytes.fromhex("158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f")
my_xonly = my_private_key.point.xonly()
tap_script_1 = TapScript([my_xonly, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_2 = TapLeaf(tap_script_2)
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_leaf_3 = TapLeaf(tap_script_3)
prev_tx = bytes.fromhex("201409034581136743bd7fd0a63f659d8142f1a41031d5a3c96bbe72135ab8a2")
prev_index = 0
target_address = "tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg"
fee = 500
# create the two branches needed (leaf 1, leaf 2), (branch 1, leaf 3)
tap_branch_1 = TapBranch(tap_leaf_1, tap_leaf_2)
tap_branch_2 = TapBranch(tap_branch_1, tap_leaf_3)
# create a transaction input
tx_in = TxIn(prev_tx, prev_index)
# calculate the target amount
target_amount = tx_in.value(network="signet") - fee
# calculate the target script
target_script = address_to_script_pubkey(target_address)
# create a transaction output
tx_out = TxOut(target_amount, target_script)
# create a transaction, segwit=True and network="signet"
tx_obj = Tx(1, [tx_in], [tx_out], network="signet", segwit=True)
# create the control block with variables left over from the exercise 2 times ago
cb = tap_branch_2.control_block(internal_pubkey, tap_leaf_1)
tx_in.witness = Witness([tap_script_1.raw_serialize(), cb.serialize()])
msg = tx_obj.sig_hash(0)
sig = my_private_key.sign_schnorr(msg).serialize()
tx_in.witness.items.insert(0, sig)
print(tx_obj.verify())
# print the serialized hex
print(tx_obj.serialize().hex())